<font size="5">**Initial Data Cleaning and Selection**</font>

<font size="3">**Load data**

In [35]:
import numpy as np # linear algebra
import pandas as pd # data processing

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# save filepath to variable for easier access
opioid_filepath = '../data/VSRR_Provisional_Drug_Overdose_Death_Counts.csv'

# read the data and store data in DataFrame 
opioid_data = pd.read_csv(opioid_filepath)

<font size="3">**Initial look at data**

In [36]:
opioid_data.head()

,State,Year,Month,Period,Indicator,Data Value,Percent Complete,Percent Pending Investigation,State Name,Footnote,Footnote Symbol,Predicted Value
0,AK,2015,April,12 month-ending,"Natural, semi-synthetic, & synthetic opioids, ...",NaN,100,0.0,Alaska,Numbers may differ from published reports usin...,**,NaN
1,AK,2015,April,12 month-ending,Natural & semi-synthetic opioids (T40.2),NaN,100,0.0,Alaska,Numbers may differ from published reports usin...,**,NaN
2,AK,2015,April,12 month-ending,"Natural & semi-synthetic opioids, incl. methad...",NaN,100,0.0,Alaska,Numbers may differ from published reports usin...,**,NaN
3,AK,2015,April,12 month-ending,Number of Deaths,4133.0,100,0.0,Alaska,Numbers may differ from published reports usin...,**,NaN
4,AK,2015,April,12 month-ending,"Opioids (T40.0-T40.4,T40.6)",NaN,100,0.0,Alaska,Numbers may differ from published reports usin...,**,NaN


In [37]:
opioid_data.tail()

,State,Year,Month,Period,Indicator,Data Value,Percent Complete,Percent Pending Investigation,State Name,Footnote,Footnote Symbol,Predicted Value
41620,YC,2021,March,12 month-ending,Cocaine (T40.5),908.00,100,0.258858,New York City,Underreported due to incomplete data.,*,937
41621,YC,2021,March,12 month-ending,Percent with drugs specified,99.06,100,0.258858,New York City,Underreported due to incomplete data.,*,NaN
41622,YC,2021,March,12 month-ending,"Natural, semi-synthetic, & synthetic opioids, ...",1853.00,100,0.258858,New York City,Underreported due to incomplete data.,*,"1,914"
41623,YC,2021,March,12 month-ending,"Natural & semi-synthetic opioids, incl. methad...",672.00,100,0.258858,New York City,Underreported due to incomplete data.,*,693
41624,YC,2021,March,12 month-ending,"Synthetic opioids, excl. methadone (T40.4)",1718.00,100,0.258858,New York City,Underreported due to incomplete data.,*,"1,778"


The data includes New York City, which isn't a state, so we'll need to take another look at the full data later.

In [38]:
opioid_data.dtypes

State                             object
Year                               int64
Month                             object
Period                            object
Indicator                         object
Data Value                       float64
Percent Complete                  object
Percent Pending Investigation    float64
State Name                        object
Footnote                          object
Footnote Symbol                   object
Predicted Value                   object
dtype: object

Here, "Year" is an integer, so we'll need to reformat this later.

In [39]:
opioid_data.shape

(41625, 12)

<font size="3">**Removing Columns**

In [40]:
# Selecting columns
opioid_features = ['State', 'Year', 'Month', 'Indicator', 'Data Value', 'State Name']
# Creating new dataframe with selected columns
selected_opioid_data = opioid_data[opioid_features]
# Reviewing new dataframe
selected_opioid_data.head()

,State,Year,Month,Indicator,Data Value,State Name
0,AK,2015,April,"Natural, semi-synthetic, & synthetic opioids, ...",NaN,Alaska
1,AK,2015,April,Natural & semi-synthetic opioids (T40.2),NaN,Alaska
2,AK,2015,April,"Natural & semi-synthetic opioids, incl. methad...",NaN,Alaska
3,AK,2015,April,Number of Deaths,4133.0,Alaska
4,AK,2015,April,"Opioids (T40.0-T40.4,T40.6)",NaN,Alaska


<font size="3">**Merging "Year" and "Month" to form a new "Date" Column**

This still leaves us with the need to change the "Year" column to a String and to merge the "Year" and "Month" together into a single Date. We then need to parse the Date together into a format the notebook can recognise as a Date.

In [41]:
# Changing all data to String
selected_opioid_data = selected_opioid_data.astype(str)

In [42]:
# Confirming the "Year" column changed to String
selected_opioid_data.dtypes

State         object
Year          object
Month         object
Indicator     object
Data Value    object
State Name    object
dtype: object

In [43]:
# Joining the "Month" and "Year" columns together
selected_opioid_data["Date"] = selected_opioid_data["Month"] + " " + selected_opioid_data["Year"]
# Taking a look at the new dataframe
selected_opioid_data.head()

,State,Year,Month,Indicator,Data Value,State Name,Date
0,AK,2015,April,"Natural, semi-synthetic, & synthetic opioids, ...",nan,Alaska,April 2015
1,AK,2015,April,Natural & semi-synthetic opioids (T40.2),nan,Alaska,April 2015
2,AK,2015,April,"Natural & semi-synthetic opioids, incl. methad...",nan,Alaska,April 2015
3,AK,2015,April,Number of Deaths,4133.0,Alaska,April 2015
4,AK,2015,April,"Opioids (T40.0-T40.4,T40.6)",nan,Alaska,April 2015


Looks good

In [44]:
# Now we will parse the Date so that the notebook can read it as an actual date
import datetime
selected_opioid_data['Date_parsed'] = pd.to_datetime(selected_opioid_data['Date'], format="%B %Y")
# Seeing how it looks
selected_opioid_data['Date_parsed'].head()

0   2015-04-01
1   2015-04-01
2   2015-04-01
3   2015-04-01
4   2015-04-01
Name: Date_parsed, dtype: datetime64[ns]

That looks good. And for the full dataframe:

In [45]:
selected_opioid_data.head()

,State,Year,Month,Indicator,Data Value,State Name,Date,Date_parsed
0,AK,2015,April,"Natural, semi-synthetic, & synthetic opioids, ...",nan,Alaska,April 2015,2015-04-01
1,AK,2015,April,Natural & semi-synthetic opioids (T40.2),nan,Alaska,April 2015,2015-04-01
2,AK,2015,April,"Natural & semi-synthetic opioids, incl. methad...",nan,Alaska,April 2015,2015-04-01
3,AK,2015,April,Number of Deaths,4133.0,Alaska,April 2015,2015-04-01
4,AK,2015,April,"Opioids (T40.0-T40.4,T40.6)",nan,Alaska,April 2015,2015-04-01


In [46]:
# Setting the Parsed Date (that the notebook can read as a date) to the index column for easier analysis later on
date_indexed_opioid_data = selected_opioid_data.set_index('Date_parsed')
# Taking a look
date_indexed_opioid_data.head()

,State,Year,Month,Indicator,Data Value,State Name,Date
Date_parsed,,,,,,,
2015-04-01,AK,2015,April,"Natural, semi-synthetic, & synthetic opioids, ...",nan,Alaska,April 2015
2015-04-01,AK,2015,April,Natural & semi-synthetic opioids (T40.2),nan,Alaska,April 2015
2015-04-01,AK,2015,April,"Natural & semi-synthetic opioids, incl. methad...",nan,Alaska,April 2015
2015-04-01,AK,2015,April,Number of Deaths,4133.0,Alaska,April 2015
2015-04-01,AK,2015,April,"Opioids (T40.0-T40.4,T40.6)",nan,Alaska,April 2015


Looks good.

Writing the clean data into a csv so we can visualize

In [47]:
# Setting the Indicator to only look at the "Number of Drug Overdose Deaths"
Num_of_deaths_data = date_indexed_opioid_data[date_indexed_opioid_data.Indicator == 'Number of Drug Overdose Deaths']
# Taking a look
Num_of_deaths_data.head()

# Selecting columns
opioid_features = ['State', 'Year', 'Month', 'Data Value', 'State Name']
# Creating new dataframe with selected columns
Num_of_deaths_data = Num_of_deaths_data[opioid_features]

Num_of_deaths_data.to_csv("../data/Clean_Opioids-in-the-us-data.csv")